# Práctica de alineamiento de secuencias

En esta práctica aprenderemos a usar el paquete DECIPHER en R para el alineamiento de múltiples secuencias. El paquete [DECIPHER](http://www2.decipher.codes/) ha sido creado por Erik Wright y está distribuído bajo licencia GPL 3 a través de Bioconductor.

En esta misma carpeta tenéis el tutorial *The Art of Multiple Sequence Alignment in R*, en el cual se basan los ejercicios de esta prácitca. El tutorial contiene mucha más información. Su figura 6 se reproduce a continuación:

![](fig6.png)

## Alineamiento de un solo gen codificante
En la carpeta `data` tienes el archivo fasta `50S_ribosomal_protein_L2.fas`, con secuencias nucloetídicas de un gen en 317 bacterias. Para introducir los datos en el espacio de trabajo de R usamos a continuación la función `readDNAStringSet()`, del paquete `Biostrings`, que está incluido en `DECIPHER`. Lee y ejecuta el bloque de código siguiente, y aségurate de comprender lo que está pasando en cada línea:

In [ ]:
suppressMessages(library('DECIPHER'))
fastaFile <- 'data/50S_ribosomal_protein_L2.fas'
dna <- readDNAStringSet(fastaFile)
dna

El objeto `dna` contiene las secuencias nucleotídicas y sus nombres. Al invocar el nombre del objeto en la línea de comandos se nos muestra un pequeño extracto del principio y del final del conjunto de secuencias. Algunas clases de objetos en R pueden parecer opacas. **Hay alguna información adicional sobre `dna` que te gustaría conocer? Sabrías explorar las funciones disponibles para trabajar con esta clase de objeto?**

### Ejercicio 1
La función para alinear las secuencias es `AlignSeqs()`. Puedes consultar la ayuda de esta función mediante `help(AlignSeqs)`. Crea un alineamiento de las secuencias de nucleótidos. Después usa la función `BrowseSeqs()` para visualizar el alineamiento, y finalmente guarda el alineamiento en formato FASTA con la función `writeXStringSet()`. **Nota 1**: No olvides usar `<-` para asignar el resultado de la función `AlignSeqs()` a un objeto nuevo. **Nota 2**: para que `BrowseSeqs()` muestre el alineamiento en este entorno de trabajo debes especificar la opción `htmlFile = <nombre del archivo html>` para crear un archivo html y después abrirlo manualmente.

## Alineamiento en espacio de aminoácidos
Puesto que los genes alineados son codificantes, podemos alinear sus traducciones a polipéptidos, que es mucho más eficaz. Además, tenemos la posibilidad de *des-traducir* el alineamiento posteriormente. Para todo ello, usamos la función `AlignTranslation()`, en la que podemos especificar el tipo de secuencias, nucleótidos o aminoácidos, que queremos ver alineadas: 

In [ ]:
alineamiento.aa <- AlignTranslation(dna, type = 'AAStringSet')
BrowseSeqs(alineamiento.aa, openURL=TRUE)

### Ejercicio 2
En el ejemplo anterior, `alineamiento.aa` contiene las secuencias de aminoácidos alineadas. Usa tú ahora la misma función, pero con la opción `type = 'DNAStringSet'` para producir el mismo alineamiento pero en nucleótidos. Después intenta comparar el resultado con el alineamiento realizado anteriormente sin traducción.

## Evitar falsas homologías
En la evolución molecular, pueden producirse **inserciones** en linajes independientes, pero en posiciones próximas de la secuencia. Como resultado, el alineamiento puede considerar homólogas regiones que en realidad no lo son, cerca de donde se encuentran los *gaps*. El paquete `DECIPHER` incluye la función `StaggerAlignment()` para intentar aliviar esta situación y mejorar la inferencia filogenética.

### Ejercicio 3
Utiliza la función `StaggerAlignment()` para re-alinear las secuencias aminoacídicas guardadas en el objeto `alineamiento.aa` y compara los dos alineamientos. Comprueba que puedes encontrar diferencias.

## Orientación de las secuencias

Algunos genes se encuentran en la cadena complementaria a la representada por la secuencia en una base de datos. Si por cualquier motivo sospechamos que algunas secuencias pueden ser las invertidas-complementarias de los genes, antes de alinear habría que corregir el sentido. Para ello, `DECIPHER` cuenta con la funcion `OrientNucleotides()`. Esta función toma una o más secuencias de referencia y revisa la semejanza de todas las demás a este grupo de referencia. Si invertir-complementar alguna secuencia hace que sea *bastante* más parecida a alguna secuencia de referencia, entonces se acepta el cambio de sentido.

Al ejecutar el bloque siguiente, sustituyes 12 secuencias por sus complementarias invertidas, sin guardar registro de qué secuencias has cambiado:

In [ ]:
muestra <- sample(1:length(dna), 12)
dna.simulado <- dna
dna.simulado[muestra] <- reverseComplement(dna.simulado[muestra])
rm(muestra)
dna.simulado

### Ejercicio 4
¿Puedes identificar las secuencias invertidas? ¿Qué estrategia usarías para comprobar si todas las secuencias codificantes de un archivo FASTA están en la misma orientación? ¿Hay alguna manera de definir un conjunto de secuencias de referencia con una orientación común? Puedes usar el bloque de código a continuación para hacer tus comprobaciones antes de discutir tus conclusiones.

## Alineamiento de genes no codificantes
En la carpeta `data` tienes también el archivo FASTA `16S_ribosomal_RNA.fas` con secuencias de RNA no codificantes. `DECIPHER` puede utilizar la predicción de estructura secundaria de los RNAs no codificantes para guiar el alineamiento de los nucleótidos. Para ello sólo es necesario que a la hora de cargar las secuencias en memoria éstas sean definidas como RNA. Entonces, la función `AlignSeqs()` lo detectará y utilizará la predicción de estructura secundaria en el alineamiento:

In [ ]:
rna <- readRNAStringSet('data/16S_ribosomal_RNA.fas')
rna

Podríamos haber leído las secuencias como DNA mediante la función `readDNAStringSet()`. Y también podemos cambiar de uno a otro tipo de secuencia. Por ejemplo:

In [ ]:
dna.16S <- DNAStringSet(rna)
dna.16S

### Ejercicio 5
Alinea las secuencias de RNA ribosomal de las dos maneras, como RNA y como DNA, y compara los resultados. **Nota**: si quieres evitar los largos mensajes producidos por `AlignSeqs()`, puedes usar la opción `verbose = FALSE`.

## Alineamiento de regiones homólogas entre genomas
Los alineamientos múltiples realizados hasta ahora son globales y asumen que la homología entre las secuencias se extiende colinearlmente de principio a fin. Otros tipos de conjuntos de secuencias (genes con intrones, proteínas multidominio, genomas enteros...) pueden contener regiones homólogas y no homólogas, así como reordenamientos de la secuencia que rompa la colinearidad: duplicaciones, inversiones y translocaciones. Para alinear estos tipos de secuencias podemos usar las funiones `FindSynteny()` y `AlignSynteny()` de `DECIPHER`.

Este ejemplo nos permite introducir un aspecto técnico de `DECIPHER` que le confiere la capacidad de alinear muchas más secuencias de las que caben en la memoria de trabajo. Para ello `DECIPHER` tiene la capacidad de trabajar con **bases de datos** locales: archivos binarios en los que se guardan las secuencias que se quieren comparar. Es posible crear una base de datos (de tipo SQLite) a partir de archivos FASTA. Para este ejercicio, usaremos una base de datos de ejemplo, ya incluida en el paquete `DECIPHER`. Contiene cinco genomas de influenzavirus A. La función `system.file()` encuentra la dirección de los archivos especificados:

In [ ]:
db <- system.file("extdata", "Influenza.sqlite", package = "DECIPHER")
db

In [ ]:
synteny <- FindSynteny(db, verbose=FALSE)
synteny

In [ ]:
pairs(synteny, boxBlocks=TRUE)

## Puntos a recordar
- ¿Cómo guardamos los resultados de un alineamiento?
- ¿De qué manera podemos evaluar la calidad de un alineamiento?
- ¿Por qué es mejor alinear aminoácidos que nucleótidos?
- ¿Mejora el alineamiento de secuencias de RNA teniendo en cuenta su estructura secundaria?